In [1]:
### Import dependencies
# Dependencies for data manipulation
import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from dateutil.relativedelta import *

# Dependencies for Databases
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
from sqlalchemy import create_engine

# Dependencies for APIs
from bs4 import BeautifulSoup
import requests
import json

# Dependencies for Webscraping
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

# Dependencies for random forest model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from joblib import dump, load

# Internal imports
from db import get_df

# Find environment variables
DATABASE_URL = os.environ.get("DATABASE_URL", None)
# sqlalchemy deprecated urls which begin with "postgres://"; now it needs to start with "postgresql://"
if DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)


In [2]:
# Get all players' name, team name, position
urlString = "https://api.myfantasyleague.com/2022/export?TYPE=players"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')
data = []
players = soup.find_all('player')
for i in range(len(players)):
    rows = [players[i].get("id"), players[i].get("name"), players[i].get("position"), players[i].get("team")]
    data.append(rows)
scrape1 = pd.DataFrame(data)
scrape1.columns=['PlayerID','Name', 'Position', 'Team']

# Get Shark Ranks
urlString = "https://api.myfantasyleague.com/2022/export?TYPE=playerRanks"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')
data = []
sharkRanks = soup.find_all('player')
for i in range(len(sharkRanks)):
    rows = [sharkRanks[i].get("id"), sharkRanks[i].get("rank")]
    data.append(rows)
shark_df = pd.DataFrame(data)
shark_df.columns=['PlayerID','SharkRank']
shark_df['SharkRank'] = shark_df['SharkRank'].astype('int32')

# Get ADP
urlString = "https://api.myfantasyleague.com/2022/export?TYPE=adp"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')
data = []
players = soup.find_all('player')
for i in range(len(players)):
    rows = [players[i].get("id"), players[i].get("averagePick")]
    data.append(rows)
adp_df = pd.DataFrame(data)
adp_df.columns=['PlayerID','ADP']
adp_df['ADP'] = adp_df['ADP'].astype('float32')

# Get player ages
# Get any player dobs who are already in the db
player_dobs = get_df('player_dobs')
# Check for any players whose ages are not already in the db
to_query_age = scrape1[~scrape1['PlayerID'].isin(player_dobs['PlayerID'])]
if len(to_query_age)>0:
    # Break player list into chunks small enough for the API server
    n = 50  #chunk row size
    list_df = [to_query_age.PlayerID[i:i+n] for i in range(0,to_query_age.PlayerID.shape[0],n)]

    for i in range(len(list_df)):
        idList = ",".join(list_df[i])

        # Get playerProfiles
        urlString = f"https://api.myfantasyleague.com/2022/export?TYPE=playerProfile&P={idList}"
        response = requests.get(urlString)
        soup = BeautifulSoup(response.content,'xml')
        data = []
        profiles = soup.find_all('playerProfile')
        players = soup.find_all('player')
        for i in range(len(profiles)):
            rows = [profiles[i].get("id"), players[i].get("dob")]
            data.append(rows)
        data_df = pd.DataFrame(data)
        age = pd.DataFrame(columns=['PlayerID', 'DOB'])
        age['PlayerID'] = data_df[0]
        age['DOB'] = data_df[1]
        player_dobs = player_dobs.append(age)

# Convert string to datetime
player_dobs['DOB'] = pd.to_datetime(player_dobs['DOB'])
# Convert DOB to Age
today = date.today()
def age(born):
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
player_dobs['Age'] = player_dobs['DOB'].apply(age)

# Merge all dfs from MyFantasyLeague API
scrape1 = scrape1.merge(player_dobs, on='PlayerID', how='left')
scrape1 = scrape1.drop(columns='DOB')
scrape1 = scrape1.merge(shark_df, on='PlayerID', how='left').merge(adp_df, on='PlayerID', how='left')
scrape1['SharkRank'].fillna(3000, inplace=True)
scrape1['ADP'].fillna(3000, inplace=True)
scrape1 = scrape1.sort_values(by=['SharkRank'])
scrape1.reset_index(inplace=True, drop=True)  

### Clean MFL data
## Select only relevant positions
scrape1 = scrape1.loc[scrape1['Position'].isin(['QB', 'WR', 'RB', 'TE', 'PK', 'Def'])]
scrape1 = scrape1.reset_index(drop=True)
## Clean Name column
to_join = scrape1['Name'].str.split(", ", n=1, expand=True)
to_join.columns = ['lname', 'fname']
to_join['Name'] = to_join['fname'] + " " + to_join['lname']
scrape1['Name'] = to_join['Name']
# Change name to Title Case
scrape1['Name'] = scrape1['Name'].str.upper()
# Drop name punctuation
scrape1['Name'] = scrape1['Name'].str.replace(".", "")
scrape1['Name'] = scrape1['Name'].str.replace(",", "")
scrape1['Name'] = scrape1['Name'].str.replace("'", "")
## Clean position column
scrape1['Position'] = scrape1['Position'].replace('Def', 'DF')
# Clean Team column
scrape1['Team'] = scrape1['Team'].replace('FA*', 'FA')
## Change column names
scrape1.columns = ['id_mfl', 'player', 'pos_mfl', 'team', 'age', 'sharkRank', 'adp']
scrape1.loc[scrape1.player=="JALEN REAGOR"]

/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/ipykernel_launcher.py:97: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


,id_mfl,player,pos_mfl,team,age,sharkRank,adp
316,14839,JALEN REAGOR,WR,MIN,23.0,895.0,3000.0


In [12]:
### scrape2 posRanks
# Set Selenium settings
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"
# scrape2 web for stats
url = f"https://www.ourlads.com/nfldepthcharts/depthcharts.aspx"
PATH = "/Applications/chromedriver"
driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
wait = WebDriverWait(driver, 20)
driver.get(url)
wait.until(EC.presence_of_element_located((By.XPATH, "//table[@id='ctl00_phContent_gvChart']")))
driver.execute_script("window.stop();")

scrape2 = pd.read_html(driver.find_element(By.XPATH, value="//table[@id='ctl00_phContent_gvChart']").get_attribute("outerHTML"))
scrape2 = scrape2[0]

In [13]:
### Clean scrape2d data
scrape2 = scrape2[['Team', 'Pos', 'Player 1', 'Player 2','Player 3', 'Player 4', 'Player 5']]

# Transform columns into rows
scrape21 = scrape2[['Team', 'Pos', 'Player 1']]
scrape21 = scrape21.rename(columns={'Player 1':'Player'})
scrape21['posRank'] = "1"

scrape22 = scrape2[['Team', 'Pos', 'Player 2']]
scrape22 = scrape22.rename(columns={'Player 2':'Player'})
scrape22['posRank'] = "2"

scrape23 = scrape2[['Team', 'Pos', 'Player 3']]
scrape23 = scrape23.rename(columns={'Player 3':'Player'})
scrape23['posRank'] = "3"

scrape24 = scrape2[['Team', 'Pos', 'Player 4']]
scrape24 = scrape24.rename(columns={'Player 4':'Player'})
scrape24['posRank'] = "4"

scrape25 = scrape2[['Team', 'Pos', 'Player 5']]
scrape25 = scrape25.rename(columns={'Player 5':'Player'})
scrape25['posRank'] = "5"

scrape2_complete = pd.concat([scrape21, scrape22, scrape23, scrape24, scrape25], axis=0, ignore_index=True)

# Clean Position column
# Select only relevant positions
posList = ['LWR', 'RWR', 'SWR', 'TE', 'QB', 'RB', 'PK', 'PR', 'KR', 'RES']
scrape2_final = scrape2_complete.loc[scrape2_complete['Pos'].isin(posList)]
# Convert WR roles to "WR"
scrape2_final['Pos'].replace(["LWR", "RWR", "SWR"], "WR", inplace=True)
scrape2_final['posRank'] = scrape2_final['Pos'] + scrape2_final['posRank']
scrape2_final = scrape2_final.reset_index(drop=True)
scrape2_final.dropna(inplace=True)
scrape2_final.drop_duplicates(subset=['Player', 'Team', 'Pos'], inplace=True)

# Create columns for KRs and PRs
krs = scrape2_final.loc[scrape2_final.Pos=='KR']
krs = krs.drop(columns=['Pos'])
krs.columns = ['Team', 'Player', 'KR']
prs = scrape2_final.loc[scrape2_final.Pos=='PR']
prs = prs.drop(columns=['Pos'])
prs.columns = ['Team', 'Player', 'PR']
# Join pr and pk scrape2s back onto main ourlads scrape2
scrape2_final = scrape2_final.merge(krs, how='left', on=['Player', 'Team']).merge(prs, how='left', on=['Player', 'Team'])
scrape2_final['KR'].fillna("NO", inplace=True)
scrape2_final['PR'].fillna("NO", inplace=True)

# Clean name column
names = scrape2_final['Player'].str.split(" ", n=2, expand=True)
names.columns = ['a', 'b', 'c']
names['a'] = names['a'].str.replace(",", "")
scrape2_final['Player'] = names['b'] + " " + names['a']
# Change to Upper Case
scrape2_final['Player'] = scrape2_final['Player'].str.upper()
# Drop punctuation
scrape2_final['Player'] = scrape2_final['Player'].str.replace(".", "")
scrape2_final['Player'] = scrape2_final['Player'].str.replace(",", "")
scrape2_final['Player'] = scrape2_final['Player'].str.replace("'", "")

# Change column names and order
scrape2_final = scrape2_final[['Player', 'Pos', 'Team', 'posRank', 'KR', 'PR']]
scrape2_final.columns = ['player', 'pos_ol', 'team', 'posRank', 'KR', 'PR']

# Remove separate rows for PRs and KRs
scrape2_final = scrape2_final.loc[(scrape2_final.pos_ol!="KR")]
scrape2_final = scrape2_final.loc[(scrape2_final.pos_ol!="PR")]

# Drop position column
scrape2_final.drop(columns=['pos_ol'], inplace=True)
scrape2_final

# Rename team abbreviations
teamDict = {
    'ARZ':'ARI', 'ATL':'ATL', 'BAL':'BAL', 'BUF':'BUF', 'CAR':'CAR', 'CHI':'CHI', 'CIN':'CIN', 'CLE':'CLE', 
    'DAL':'DAL', 'DEN':'DEN', 'DET':'DET', 'GB':'GBP', 'HOU':'HOU', 'IND':'IND', 'JAX':'JAC', 'KC':'KCC', 
    'LAC':'LAC', 'LAR':'LAR', 'LV':'LVR', 'MIA':'MIA', 'MIN':'MIN', 'NE':'NEP', 'NO':'NOS', 'NYG':'NYG', 
    'NYJ':'NYJ', 'PHI':'PHI', 'PIT':'PIT', 'SEA':'SEA', 'SF':'SFO', 'TB':'TBB', 'TEN':'TEN', 'WAS':'WAS'
    }
scrape2_final['team'] = scrape2_final['team'].map(teamDict)

/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/ipykernel_launcher.py:58: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions

In [16]:
scrape2_complete

,Team,Pos,Player,posRank
0,Arizona CardinalsUpdated: 08/31/2022 12:44PM ET,Arizona CardinalsUpdated: 08/31/2022 12:44PM ET,Arizona CardinalsUpdated: 08/31/2022 12:44PM ET,1
1,NaN,Arizona Cardinals,NaN,1
2,Offense - Arizona Cardinals,Offense - Arizona Cardinals,Offense - Arizona Cardinals,1
3,ARZ,LWR,"GREEN, A.J. U/Cin",1
4,ARZ,RWR,"Brown, Marquise T/Bal",1
...,...,...,...,...
6305,WAS,PR,NaN,5
6306,WAS,KR,NaN,5
6307,Reserves - Washington Commanders,Reserves - Washington Commanders,Reserves - Washington Commanders,5
6308,WAS,RES,"Beavers, Willie IR",5


In [17]:
scrape2_final.loc[scrape2_final.player=="JALEN REAGOR"]

,player,team,posRank,KR,PR
512,JALEN REAGOR,MIN,WR2,NO,NO


In [18]:
### Merge MyFantasyLeague data with scrape2d data
player_df = scrape1.merge(scrape2_final, how='left', on=['player', 'team'])
## Clean merged df
player_df.loc[player_df['pos_mfl']=='DF', 'posRank'] = "DF1"
player_df['KR'].fillna("NO", inplace=True)
player_df['PR'].fillna("NO", inplace=True)
## Clean posRanks
player_df['posRank'] = player_df['posRank'].map({
    'RES1':'RES',
    'RES2':'RES',
    'RES3':'RES',
    'RES4':'RES',
    'RES5':'RES',
    'QB1':'QB1', 
    'QB2':'QB2', 
    'QB3':'QB3', 
    'QB4':'QB3',
    'QB5':'QB3', 
    'RB1':'RB1', 
    'RB2':'RB2', 
    'RB3':'RB3', 
    'RB4':'RB3', 
    'RB5':'RB3',
    'WR1': 'WR1', 
    'WR2': 'WR2', 
    'WR3': 'WR3', 
    'WR4': 'WR3', 
    'WR5': 'WR3', 
    'TE1':'TE1', 
    'TE2':'TE2', 
    'TE3':'TE3', 
    'TE4':'TE3', 
    'TE5':'TE3', 
    'PK1':'PK1', 
    'PK2':'PK2', 
    'PK3':'PK3',
    'DF1':'DF1'
    })
# Create "RES/NO" column
player_df['RES'] = "NO"
player_df.loc[player_df['posRank']=="RES", 'RES'] = "RES"
player_df.loc[player_df.posRank.isna(), 'posRank'] = player_df.loc[player_df.posRank.isna(), 'pos_mfl'] + "3"
player_df.loc[player_df.posRank=="RES", 'posRank'] = player_df.loc[player_df.posRank=="RES", 'pos_mfl'] + "3"
# Specify all players are in current season
player_df['season'] = 2022

In [19]:
player_df.loc[player_df.player=="JALEN REAGOR"]

,id_mfl,player,pos_mfl,team,age,sharkRank,adp,posRank,KR,PR,RES,season
316,14839,JALEN REAGOR,WR,MIN,23.0,895.0,3000.0,WR2,NO,NO,NO,2022


In [20]:
### Get historical data
prior1 = get_df('prior1')
prior2 = get_df('prior2')


In [21]:
# Create current_df
# This will mean scraping the ff db site weekly
curr = pd.DataFrame(player_df['player'])
colList = ['gamesPlayed',
    'passA', 'passC', 'passY', 'passT', 'passI', 'pass2', 
    'rushA', 'rushY','rushT', 'rush2', 
    'recC', 'recY', 'recT', 'rec2', 'fum', 
    'XPA', 'XPM','FGA', 'FGM', 'FG50', 
    'defSack', 'defI', 'defSaf', 'defFum', 'defBlk','defT', 'defPtsAgainst', 'defPassYAgainst', 'defRushYAgainst','defYdsAgainst'
]
cols = pd.DataFrame(columns=colList)
curr = curr.merge(cols, how='left', left_index=True, right_index=True)
curr.fillna(0, inplace=True)

# Rename all columns in curr
colList = [(x + "_curr") for x in list(curr.columns)]
curr.columns = colList
curr = curr.rename(columns={
       'player_curr':'player',
       })
curr

,player,gamesPlayed_curr,passA_curr,passC_curr,passY_curr,passT_curr,passI_curr,pass2_curr,rushA_curr,rushY_curr,...,defSack_curr,defI_curr,defSaf_curr,defFum_curr,defBlk_curr,defT_curr,defPtsAgainst_curr,defPassYAgainst_curr,defRushYAgainst_curr,defYdsAgainst_curr
0,JONATHAN TAYLOR,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AUSTIN EKELER,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CHRISTIAN MCCAFFREY,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,DALVIN COOK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,DERRICK HENRY,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,OJ HOWARD,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1096,CHAD HANSEN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1097,GEHRIG DIETER,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1098,TAYWAN TAYLOR,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# Merge playerdf, currentdf, prior1, and prior2
player_df = player_df.merge(curr, how='left', on='player').merge(prior1, how='left', on='player').merge(prior2, how='left', on='player')
# Fill data for players who do not have prior data
player_df.fillna(0, inplace=True)

In [23]:
player_df.drop_duplicates(subset=['player', 'pos_mfl'], inplace=True)

In [24]:
player_df.loc[player_df.player=="JALEN REAGOR"]

,id_mfl,player,pos_mfl,team,age,sharkRank,adp,posRank,KR,PR,...,defSack_prior2,defI_prior2,defSaf_prior2,defFum_prior2,defBlk_prior2,defT_prior2,defPtsAgainst_prior2,defPassYAgainst_prior2,defRushYAgainst_prior2,defYdsAgainst_prior2
317,14839,JALEN REAGOR,WR,MIN,23.0,895.0,3000.0,WR2,NO,NO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# Get schedule
schedule = get_df('schedule')
# Merge in opponents
player_df = player_df.merge(schedule, how='left', on='team')
player_df.loc[player_df.player=="JALEN REAGOR"]

,id_mfl,player,pos_mfl,team,age,sharkRank,adp,posRank,KR,PR,...,defSaf_prior2,defFum_prior2,defBlk_prior2,defT_prior2,defPtsAgainst_prior2,defPassYAgainst_prior2,defRushYAgainst_prior2,defYdsAgainst_prior2,week,opponent
5356,14839,JALEN REAGOR,WR,MIN,23.0,895.0,3000.0,WR2,NO,NO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,PHI
5357,14839,JALEN REAGOR,WR,MIN,23.0,895.0,3000.0,WR2,NO,NO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,NOS
5358,14839,JALEN REAGOR,WR,MIN,23.0,895.0,3000.0,WR2,NO,NO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,MIA
5359,14839,JALEN REAGOR,WR,MIN,23.0,895.0,3000.0,WR2,NO,NO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,WAS
5360,14839,JALEN REAGOR,WR,MIN,23.0,895.0,3000.0,WR2,NO,NO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,BUF
5361,14839,JALEN REAGOR,WR,MIN,23.0,895.0,3000.0,WR2,NO,NO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,DET
5362,14839,JALEN REAGOR,WR,MIN,23.0,895.0,3000.0,WR2,NO,NO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,GBP
5363,14839,JALEN REAGOR,WR,MIN,23.0,895.0,3000.0,WR2,NO,NO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,CHI
5364,14839,JALEN REAGOR,WR,MIN,23.0,895.0,3000.0,WR2,NO,NO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,GBP
5365,14839,JALEN REAGOR,WR,MIN,23.0,895.0,3000.0,WR2,NO,NO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,DET


In [26]:
# Rename position column in player_df
player_df.rename(columns={'pos_mfl':'pos'}, inplace=True)

In [27]:
# Get opponent historical data
# select only defenses
allDef = player_df.loc[player_df['pos']=='DF']
allDef

,id_mfl,player,pos,team,age,sharkRank,adp,posRank,KR,PR,...,defSaf_prior2,defFum_prior2,defBlk_prior2,defT_prior2,defPtsAgainst_prior2,defPassYAgainst_prior2,defRushYAgainst_prior2,defYdsAgainst_prior2,week,opponent
2516,0528,LOS ANGELES RAMS,DF,LAR,0.0,266.0,111.949997,DF1,NO,NO,...,0.03125,0.43750,0.03125,0.15625,20.03125,213.9375,96.21875,310.15625,3.0,ARI
2517,0528,LOS ANGELES RAMS,DF,LAR,0.0,266.0,111.949997,DF1,NO,NO,...,0.03125,0.43750,0.03125,0.15625,20.03125,213.9375,96.21875,310.15625,4.0,SFO
2518,0528,LOS ANGELES RAMS,DF,LAR,0.0,266.0,111.949997,DF1,NO,NO,...,0.03125,0.43750,0.03125,0.15625,20.03125,213.9375,96.21875,310.15625,9.0,TBB
2519,0528,LOS ANGELES RAMS,DF,LAR,0.0,266.0,111.949997,DF1,NO,NO,...,0.03125,0.43750,0.03125,0.15625,20.03125,213.9375,96.21875,310.15625,11.0,NOS
2520,0528,LOS ANGELES RAMS,DF,LAR,0.0,266.0,111.949997,DF1,NO,NO,...,0.03125,0.43750,0.03125,0.15625,20.03125,213.9375,96.21875,310.15625,12.0,KCC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5572,0513,LAS VEGAS RAIDERS,DF,LVR,0.0,983.0,169.960007,DF1,NO,NO,...,0.03125,0.40625,0.06250,0.00000,27.65625,238.9375,120.96875,359.90625,10.0,IND
5573,0513,LAS VEGAS RAIDERS,DF,LVR,0.0,983.0,169.960007,DF1,NO,NO,...,0.03125,0.40625,0.06250,0.00000,27.65625,238.9375,120.96875,359.90625,13.0,LAC
5574,0513,LAS VEGAS RAIDERS,DF,LVR,0.0,983.0,169.960007,DF1,NO,NO,...,0.03125,0.40625,0.06250,0.00000,27.65625,238.9375,120.96875,359.90625,15.0,NEP
5575,0513,LAS VEGAS RAIDERS,DF,LVR,0.0,983.0,169.960007,DF1,NO,NO,...,0.03125,0.40625,0.06250,0.00000,27.65625,238.9375,120.96875,359.90625,17.0,SFO


In [28]:

# Get current defensive scores
currDef = allDef.copy()
# Select only relevant columns
currDef = currDef[['team', 'week',
       'defSack_curr', 'defI_curr',
       'defSaf_curr', 'defFum_curr', 'defBlk_curr',
       'defT_curr', 'defPtsAgainst_curr', 'defPassYAgainst_curr',
       'defRushYAgainst_curr', 'defYdsAgainst_curr']]

# Get prior defensive scores
priorDef = allDef.copy()
# Select only relevant columns
priorDef = priorDef[['team', 'week',
       'defSack_prior1', 'defI_prior1',
       'defSaf_prior1', 'defFum_prior1', 'defBlk_prior1',
       'defT_prior1', 'defPtsAgainst_prior1', 'defPassYAgainst_prior1',
       'defRushYAgainst_prior1', 'defYdsAgainst_prior1']]
# Merge the two defensive dfs
allDef = currDef.merge(priorDef, how='left', on=['team', 'week'])

# Rename all columns in allDef
colList = [(x + "_opp") for x in list(allDef.columns)]
allDef.columns = colList
allDef = allDef.rename(columns={
       'team_opp':'opponent',
       'week_opp':'week'
       })

In [29]:
# Connect opponents to defenses
player_df = player_df.merge(allDef, how='left', on=['opponent', 'week'])
print(len(player_df))

11116


In [30]:

player_df = player_df[[
    'id_mfl',
    'season',
    'week',
    'team',
    'player',
    'age',
    'sharkRank', 
    'adp',
    'KR',
    'PR',
    'RES',
    'pos',
    'posRank',
    'opponent',
    'passA_curr',
    'passC_curr',
    'passY_curr',
    'passT_curr',
    'passI_curr',
    'pass2_curr',
    'rushA_curr',
    'rushY_curr',
    'rushT_curr',
    'rush2_curr',
    'recC_curr',
    'recY_curr',
    'recT_curr',
    'rec2_curr',
    'fum_curr',
    'XPA_curr',
    'XPM_curr',
    'FGA_curr',
    'FGM_curr',
    'FG50_curr',
    'defSack_curr',
    'defI_curr',
    'defSaf_curr',
    'defFum_curr',
    'defBlk_curr',
    'defT_curr',
    'defPtsAgainst_curr',
    'defPassYAgainst_curr',
    'defRushYAgainst_curr',
    'defYdsAgainst_curr',
    'gamesPlayed_curr',
    'gamesPlayed_prior1',
    'passA_prior1',
    'passC_prior1',
    'passY_prior1',
    'passT_prior1',
    'passI_prior1',
    'pass2_prior1',
    'rushA_prior1',
    'rushY_prior1',
    'rushT_prior1',
    'rush2_prior1',
    'recC_prior1',
    'recY_prior1',
    'recT_prior1',
    'rec2_prior1',
    'fum_prior1',
    'XPA_prior1',
    'XPM_prior1',
    'FGA_prior1',
    'FGM_prior1',
    'FG50_prior1',
    'defSack_prior1',
    'defI_prior1',
    'defSaf_prior1',
    'defFum_prior1',
    'defBlk_prior1',
    'defT_prior1',
    'defPtsAgainst_prior1',
    'defPassYAgainst_prior1',
    'defRushYAgainst_prior1',
    'defYdsAgainst_prior1',
    'gamesPlayed_prior2',
    'passA_prior2',
    'passC_prior2',
    'passY_prior2',
    'passT_prior2',
    'passI_prior2',
    'pass2_prior2',
    'rushA_prior2',
    'rushY_prior2',
    'rushT_prior2',
    'rush2_prior2',
    'recC_prior2',
    'recY_prior2',
    'recT_prior2',
    'rec2_prior2',
    'fum_prior2',
    'XPA_prior2',
    'XPM_prior2',
    'FGA_prior2',
    'FGM_prior2',
    'FG50_prior2',
    'defSack_prior2',
    'defI_prior2',
    'defSaf_prior2',
    'defFum_prior2',
    'defBlk_prior2',
    'defT_prior2',
    'defPtsAgainst_prior2',
    'defPassYAgainst_prior2',
    'defRushYAgainst_prior2',
    'defYdsAgainst_prior2',
    'defSack_curr_opp',
    'defI_curr_opp',
    'defSaf_curr_opp',
    'defFum_curr_opp',
    'defBlk_curr_opp',
    'defT_curr_opp',
    'defPtsAgainst_curr_opp',
    'defPassYAgainst_curr_opp',
    'defRushYAgainst_curr_opp',
    'defYdsAgainst_curr_opp',
    'defSack_prior1_opp',
    'defI_prior1_opp',
    'defSaf_prior1_opp',
    'defFum_prior1_opp',
    'defBlk_prior1_opp',
    'defT_prior1_opp',
    'defPtsAgainst_prior1_opp',
    'defPassYAgainst_prior1_opp',
    'defRushYAgainst_prior1_opp',
    'defYdsAgainst_prior1_opp']]

In [31]:
player_df.loc[player_df.player=="JALEN REAGOR"]

,id_mfl,season,week,team,player,age,sharkRank,adp,KR,PR,...,defSack_prior1_opp,defI_prior1_opp,defSaf_prior1_opp,defFum_prior1_opp,defBlk_prior1_opp,defT_prior1_opp,defPtsAgainst_prior1_opp,defPassYAgainst_prior1_opp,defRushYAgainst_prior1_opp,defYdsAgainst_prior1_opp
5356,14839,2022,2.0,MIN,JALEN REAGOR,23.0,895.0,3000.0,NO,NO,...,1.7500,0.7500,0.0000,0.2500,0.1250,0.3125,20.8750,215.7500,103.9375,319.6875
5357,14839,2022,4.0,MIN,JALEN REAGOR,23.0,895.0,3000.0,NO,NO,...,2.6875,1.0625,0.0000,0.3125,0.1250,0.1250,19.6875,226.6250,95.4375,322.0625
5358,14839,2022,6.0,MIN,JALEN REAGOR,23.0,895.0,3000.0,NO,NO,...,2.8750,0.8125,0.0000,0.6250,0.0625,0.2500,21.8125,226.6250,108.3125,334.9375
5359,14839,2022,9.0,MIN,JALEN REAGOR,23.0,895.0,3000.0,NO,NO,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
5360,14839,2022,10.0,MIN,JALEN REAGOR,23.0,895.0,3000.0,NO,NO,...,2.0625,1.1875,0.0000,0.6875,0.0625,0.0625,17.4375,172.8750,113.6250,286.5000
5361,14839,2022,14.0,MIN,JALEN REAGOR,23.0,895.0,3000.0,NO,NO,...,1.6250,0.5625,0.0000,0.4375,0.0625,0.0000,27.3125,244.0000,135.8750,379.8750
5362,14839,2022,17.0,MIN,JALEN REAGOR,23.0,895.0,3000.0,NO,NO,...,2.3750,1.1250,0.0000,0.5000,0.0625,0.1250,20.8750,213.6875,109.7500,323.4375
5363,14839,2022,18.0,MIN,JALEN REAGOR,23.0,895.0,3000.0,NO,NO,...,2.8750,0.5000,0.0625,0.5000,0.0000,0.1875,23.5000,189.3750,126.4375,315.8125
5364,14839,2022,1.0,MIN,JALEN REAGOR,23.0,895.0,3000.0,NO,NO,...,2.3750,1.1250,0.0000,0.5000,0.0625,0.1250,20.8750,213.6875,109.7500,323.4375
5365,14839,2022,3.0,MIN,JALEN REAGOR,23.0,895.0,3000.0,NO,NO,...,1.6250,0.5625,0.0000,0.4375,0.0625,0.0000,27.3125,244.0000,135.8750,379.8750


In [32]:
labels = [
    'passA', 'passC', 'passY', 'passT', 'passI', 'pass2', 
    'rushA', 'rushY', 'rushT', 'rush2', 
    'recC', 'recY', 'recT', 'rec2', 'fum', 
    'XPA', 'XPM', 'FGA', 'FGM', 'FG50', 
    'defSack', 'defI', 'defSaf', 'defFum', 'defBlk', 'defT', 
    'defPtsAgainst', 'defPassYAgainst', 'defRushYAgainst', 'defYdsAgainst'  
]

features = [
    'week', 'age', 
    'passA_curr', 'passC_curr', 'passY_curr', 'passT_curr', 'passI_curr', 'pass2_curr', 
    'rushA_curr', 'rushY_curr', 'rushT_curr', 'rush2_curr', 
    'recC_curr', 'recY_curr', 'recT_curr', 'rec2_curr', 'fum_curr', 
    'XPA_curr', 'XPM_curr', 'FGA_curr', 'FGM_curr', 'FG50_curr', 
    'defSack_curr', 'defI_curr', 'defSaf_curr', 'defFum_curr', 'defBlk_curr', 'defT_curr', 
    'defPtsAgainst_curr', 'defPassYAgainst_curr', 'defRushYAgainst_curr', 'defYdsAgainst_curr', 
    'gamesPlayed_curr', 
    'gamesPlayed_prior1', 
    'passA_prior1', 'passC_prior1', 'passY_prior1', 'passT_prior1', 'passI_prior1', 'pass2_prior1', 
    'rushA_prior1', 'rushY_prior1', 'rushT_prior1', 'rush2_prior1', 
    'recC_prior1', 'recY_prior1', 'recT_prior1', 'rec2_prior1', 'fum_prior1', 
    'XPA_prior1', 'XPM_prior1', 'FGA_prior1', 'FGM_prior1', 'FG50_prior1', 
    'defSack_prior1', 'defI_prior1', 'defSaf_prior1', 'defFum_prior1', 'defBlk_prior1', 'defT_prior1', 
    'defPtsAgainst_prior1', 'defPassYAgainst_prior1', 'defRushYAgainst_prior1', 'defYdsAgainst_prior1', 
    'gamesPlayed_prior2', 
    'passA_prior2', 'passC_prior2', 'passY_prior2', 'passT_prior2', 'passI_prior2', 'pass2_prior2', 
    'rushA_prior2', 'rushY_prior2', 'rushT_prior2', 'rush2_prior2', 
    'recC_prior2', 'recY_prior2', 'recT_prior2', 'rec2_prior2', 'fum_prior2', 
    'XPA_prior2', 'XPM_prior2', 'FGA_prior2', 'FGM_prior2', 'FG50_prior2', 
    'defSack_prior2', 'defI_prior2', 'defSaf_prior2', 'defFum_prior2', 'defBlk_prior2', 'defT_prior2', 
    'defPtsAgainst_prior2', 'defPassYAgainst_prior2', 'defRushYAgainst_prior2', 'defYdsAgainst_prior2', 
    'defSack_curr_opp', 'defI_curr_opp', 'defSaf_curr_opp', 'defFum_curr_opp', 'defBlk_curr_opp', 'defT_curr_opp', 
    'defPtsAgainst_curr_opp', 'defPassYAgainst_curr_opp', 'defRushYAgainst_curr_opp', 'defYdsAgainst_curr_opp', 
    'defSack_prior1_opp', 'defI_prior1_opp', 'defSaf_prior1_opp', 'defFum_prior1_opp', 'defBlk_prior1_opp', 'defT_prior1_opp', 
    'defPtsAgainst_prior1_opp', 'defPassYAgainst_prior1_opp', 'defRushYAgainst_prior1_opp', 'defYdsAgainst_prior1_opp', 
    'pos', 'posRank'
]


In [33]:
### WR Predictions
# Read player model and ages
xl2 = player_df.copy()
# Select only one player position
xl2 = xl2.loc[xl2.posRank.isin(['WR1', 'WR2', 'WR3'])]
xl2 = xl2.loc[xl2.pos=='WR']
xl2 = xl2.dropna()
xl2.reset_index(inplace=True, drop=True)

# Select features
X = xl2[features]
header = xl2[[
    'id_mfl',
    'season',
    'week',
    'team',
    'player',
    'age',
    'sharkRank', 
    'adp',
    'KR',
    'PR',
    'RES',
    'pos',
    'posRank',
    'opponent'
]]

# Encode categorical features
X = pd.get_dummies(X, columns = ['pos', 'posRank'])

#load saved model
regressor = load('models/rfmodel_WR1.joblib')

# Run model
y_pred = regressor.predict(X)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = labels
y_pred

# Calculate FANTASY scores
# Define scoring multiplier based on league settings
multiplier = [
    0,0,.04,4,-2,2,.1,.1,6,2,.25,.1,6,2,-2,0,1,0,3,5,1,2,2,2,1.5,6,0,0,0,0,1,1
]
# Define bins for defensive PointsAgainst and YardsAgainst based on MFL scoring categories
binList_defPts = [-5,0,6,13,17,21,27,34,45,59,99]
binList_defYds = [0,274,324,375,425,999]
# Define correlating scores for defensive PointsAgainst and YardsAgainst based on league settings
ptList_defPts = [10,8,7,5,3,2,0,-1,-3,-5]
ptList_defYds = [5,2,0,-2,-5]
# Bin and cut the defensive predictions
y_pred['defPtsBin'] = pd.cut(y_pred['defPtsAgainst'], bins=binList_defPts, include_lowest=True, labels=ptList_defPts)
y_pred['defYdsBin'] = pd.cut(y_pred['defYdsAgainst'], bins=binList_defYds, include_lowest=True, labels=ptList_defYds)
# Merge predictions with header columns so we know the players' position
a_pred = header.merge(y_pred, left_index=True, right_index=True)
# Assign value of zero to all non-defensive players' bins
a_pred.loc[a_pred['pos']!='DF', 'defPtsBin'] = 0
a_pred.loc[a_pred['pos']!='DF', 'defYdsBin'] = 0
# Drop the header columns again
a_pred = a_pred.drop(columns=['id_mfl', 'week','season','team','player','age','sharkRank','adp','pos','KR','PR','RES','posRank','opponent'])
# Create function to apply scoring multiplier
def multer(row):
    return row.multiply(multiplier)
# Apply scoring multiplier to predictions
c = a_pred.apply(multer, axis=1)
c = c.apply(np.sum, axis=1)
c = pd.DataFrame(c, columns=['pred'])

# Merge header columns with predictions
WRdf = header.merge(c, left_index=True, right_index=True)

In [34]:
### RB Predictions
# Read player model and ages
xl2 = player_df.copy()
# Select only one player position
xl2 = xl2.loc[xl2.posRank.isin(['RB1', 'RB2', 'RB3'])]
xl2 = xl2.loc[xl2.pos=='RB']
xl2 = xl2.dropna()
xl2.reset_index(inplace=True, drop=True)

# Select features
X = xl2[features]
header = xl2[[
    'id_mfl',
    'season',
    'week',
    'team',
    'player',
    'age',
    'sharkRank', 
    'adp',
    'KR',
    'PR',
    'RES',
    'pos',
    'posRank',
    'opponent'
]]

# Encode categorical features
X = pd.get_dummies(X, columns = ['pos', 'posRank'])

#load saved model
regressor = load('models/rfmodel_RB1.joblib')

# Run model
y_pred = regressor.predict(X)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = labels
y_pred

# Calculate FANTASY scores
# Define scoring multiplier based on league settings
multiplier = [
    0,0,.04,4,-2,2,.1,.1,6,2,.25,.1,6,2,-2,0,1,0,3,5,1,2,2,2,1.5,6,0,0,0,0,1,1
]
# Define bins for defensive PointsAgainst and YardsAgainst based on MFL scoring categories
binList_defPts = [-5,0,6,13,17,21,27,34,45,59,99]
binList_defYds = [0,274,324,375,425,999]
# Define correlating scores for defensive PointsAgainst and YardsAgainst based on league settings
ptList_defPts = [10,8,7,5,3,2,0,-1,-3,-5]
ptList_defYds = [5,2,0,-2,-5]
# Bin and cut the defensive predictions
y_pred['defPtsBin'] = pd.cut(y_pred['defPtsAgainst'], bins=binList_defPts, include_lowest=True, labels=ptList_defPts)
y_pred['defYdsBin'] = pd.cut(y_pred['defYdsAgainst'], bins=binList_defYds, include_lowest=True, labels=ptList_defYds)
# Merge predictions with header columns so we know the players' position
a_pred = header.merge(y_pred, left_index=True, right_index=True)
# Assign value of zero to all non-defensive players' bins
a_pred.loc[a_pred['pos']!='DF', 'defPtsBin'] = 0
a_pred.loc[a_pred['pos']!='DF', 'defYdsBin'] = 0
# Drop the header columns again
a_pred = a_pred.drop(columns=['id_mfl', 'week','season','team','player','age','sharkRank','adp','pos','KR','PR','RES','posRank','opponent'])
# Create function to apply scoring multiplier
def multer(row):
    return row.multiply(multiplier)
# Apply scoring multiplier to predictions
c = a_pred.apply(multer, axis=1)
c = c.apply(np.sum, axis=1)
c = pd.DataFrame(c, columns=['pred'])

# Merge header columns with predictions
RBdf = header.merge(c, left_index=True, right_index=True)

In [35]:
### QB Predictions
# Read player model and ages
xl2 = player_df.copy()
# Select only one player position
xl2 = xl2.loc[xl2.posRank.isin(['QB1', 'QB2', 'QB3'])]
xl2 = xl2.loc[xl2.pos=='QB']
xl2 = xl2.dropna()
xl2.reset_index(inplace=True, drop=True)

# Select features
X = xl2[features]
header = xl2[[
    'id_mfl',
    'season',
    'week',
    'team',
    'player',
    'age',
    'sharkRank', 
    'adp',
    'KR',
    'PR',
    'RES',
    'pos',
    'posRank',
    'opponent'
]]

# Encode categorical features
X = pd.get_dummies(X, columns = ['pos', 'posRank'])

#load saved model
regressor = load('models/rfmodel_QB1.joblib')

# Run model
y_pred = regressor.predict(X)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = labels
y_pred

# Calculate FANTASY scores
# Define scoring multiplier based on league settings
multiplier = [
    0,0,.04,4,-2,2,.1,.1,6,2,.25,.1,6,2,-2,0,1,0,3,5,1,2,2,2,1.5,6,0,0,0,0,1,1
]
# Define bins for defensive PointsAgainst and YardsAgainst based on MFL scoring categories
binList_defPts = [-5,0,6,13,17,21,27,34,45,59,99]
binList_defYds = [0,274,324,375,425,999]
# Define correlating scores for defensive PointsAgainst and YardsAgainst based on league settings
ptList_defPts = [10,8,7,5,3,2,0,-1,-3,-5]
ptList_defYds = [5,2,0,-2,-5]
# Bin and cut the defensive predictions
y_pred['defPtsBin'] = pd.cut(y_pred['defPtsAgainst'], bins=binList_defPts, include_lowest=True, labels=ptList_defPts)
y_pred['defYdsBin'] = pd.cut(y_pred['defYdsAgainst'], bins=binList_defYds, include_lowest=True, labels=ptList_defYds)
# Merge predictions with header columns so we know the players' position
a_pred = header.merge(y_pred, left_index=True, right_index=True)
# Assign value of zero to all non-defensive players' bins
a_pred.loc[a_pred['pos']!='DF', 'defPtsBin'] = 0
a_pred.loc[a_pred['pos']!='DF', 'defYdsBin'] = 0
# Drop the header columns again
a_pred = a_pred.drop(columns=['id_mfl', 'week','season','team','player','age','sharkRank','adp','pos','KR','PR','RES','posRank','opponent'])
# Create function to apply scoring multiplier
def multer(row):
    return row.multiply(multiplier)
# Apply scoring multiplier to predictions
c = a_pred.apply(multer, axis=1)
c = c.apply(np.sum, axis=1)
c = pd.DataFrame(c, columns=['pred'])

# Merge header columns with predictions
QBdf = header.merge(c, left_index=True, right_index=True)

In [36]:
### TE Predictions
# Read player model and ages
xl2 = player_df.copy()
# Select only one player position
xl2 = xl2.loc[xl2.posRank.isin(['TE1', 'TE2', 'TE3'])]
xl2 = xl2.loc[xl2.pos=='TE']
xl2 = xl2.dropna()
xl2.reset_index(inplace=True, drop=True)

# Select features
X = xl2[features]
header = xl2[[
    'id_mfl',
    'season',
    'week',
    'team',
    'player',
    'age',
    'sharkRank', 
    'adp',
    'KR',
    'PR',
    'RES',
    'pos',
    'posRank',
    'opponent'
]]

# Encode categorical features
X = pd.get_dummies(X, columns = ['pos', 'posRank'])

#load saved model
regressor = load('models/rfmodel_TE1.joblib')

# Run model
y_pred = regressor.predict(X)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = labels
y_pred

# Calculate FANTASY scores
# Define scoring multiplier based on league settings
multiplier = [
    0,0,.04,4,-2,2,.1,.1,6,2,.25,.1,6,2,-2,0,1,0,3,5,1,2,2,2,1.5,6,0,0,0,0,1,1
]
# Define bins for defensive PointsAgainst and YardsAgainst based on MFL scoring categories
binList_defPts = [-5,0,6,13,17,21,27,34,45,59,99]
binList_defYds = [0,274,324,375,425,999]
# Define correlating scores for defensive PointsAgainst and YardsAgainst based on league settings
ptList_defPts = [10,8,7,5,3,2,0,-1,-3,-5]
ptList_defYds = [5,2,0,-2,-5]
# Bin and cut the defensive predictions
y_pred['defPtsBin'] = pd.cut(y_pred['defPtsAgainst'], bins=binList_defPts, include_lowest=True, labels=ptList_defPts)
y_pred['defYdsBin'] = pd.cut(y_pred['defYdsAgainst'], bins=binList_defYds, include_lowest=True, labels=ptList_defYds)
# Merge predictions with header columns so we know the players' position
a_pred = header.merge(y_pred, left_index=True, right_index=True)
# Assign value of zero to all non-defensive players' bins
a_pred.loc[a_pred['pos']!='DF', 'defPtsBin'] = 0
a_pred.loc[a_pred['pos']!='DF', 'defYdsBin'] = 0
# Drop the header columns again
a_pred = a_pred.drop(columns=['id_mfl', 'week','season','team','player','age','sharkRank','adp','pos','KR','PR','RES','posRank','opponent'])
# Create function to apply scoring multiplier
def multer(row):
    return row.multiply(multiplier)
# Apply scoring multiplier to predictions
c = a_pred.apply(multer, axis=1)
c = c.apply(np.sum, axis=1)
c = pd.DataFrame(c, columns=['pred'])

# Merge header columns with predictions
TEdf = header.merge(c, left_index=True, right_index=True)

In [56]:
### PK Predictions
# Read player model and ages
xl2 = player_df.copy()
# Select only one player position
xl2 = xl2.loc[xl2.posRank.isin(['PK1', 'PK2', 'PK3'])]
xl2 = xl2.loc[xl2.pos=='PK']
xl2 = xl2.dropna()
xl2.reset_index(inplace=True, drop=True)

# Select features
X = xl2[features]
header = xl2[[
    'id_mfl',
    'season',
    'week',
    'team',
    'player',
    'age',
    'sharkRank', 
    'adp',
    'KR',
    'PR',
    'RES',
    'pos',
    'posRank',
    'opponent'
]]

# Encode categorical features
X = pd.get_dummies(X, columns = ['pos', 'posRank'])

# Check if there were the correct number of posRanks in the dataset
for rank in ["posRank_PK1", "posRank_PK2", "posRank_PK3"]:
    if rank not in list(X.columns):
        X[rank] = 0

# Make sure we have the necessary columns
X = X[['week', 'age', 'passA_curr', 'passC_curr', 'passY_curr', 'passT_curr', 'passI_curr', 'pass2_curr', 'rushA_curr', 'rushY_curr', 
'rushT_curr', 'rush2_curr', 'recC_curr', 'recY_curr', 'recT_curr', 'rec2_curr', 'fum_curr', 'XPA_curr', 'XPM_curr', 'FGA_curr', 'FGM_curr', 
'FG50_curr', 'defSack_curr', 'defI_curr', 'defSaf_curr', 'defFum_curr', 'defBlk_curr', 'defT_curr', 'defPtsAgainst_curr', 
'defPassYAgainst_curr', 'defRushYAgainst_curr', 'defYdsAgainst_curr', 'gamesPlayed_curr', 'gamesPlayed_prior1', 'passA_prior1', 
'passC_prior1', 'passY_prior1', 'passT_prior1', 'passI_prior1', 'pass2_prior1', 'rushA_prior1', 'rushY_prior1', 'rushT_prior1', 
'rush2_prior1', 'recC_prior1', 'recY_prior1', 'recT_prior1', 'rec2_prior1', 'fum_prior1', 'XPA_prior1', 'XPM_prior1', 'FGA_prior1', 
'FGM_prior1', 'FG50_prior1', 'defSack_prior1', 'defI_prior1', 'defSaf_prior1', 'defFum_prior1', 'defBlk_prior1', 'defT_prior1', 
'defPtsAgainst_prior1', 'defPassYAgainst_prior1', 'defRushYAgainst_prior1', 'defYdsAgainst_prior1', 'gamesPlayed_prior2', 'passA_prior2', 
'passC_prior2', 'passY_prior2', 'passT_prior2', 'passI_prior2', 'pass2_prior2', 'rushA_prior2', 'rushY_prior2', 'rushT_prior2', 
'rush2_prior2', 'recC_prior2', 'recY_prior2', 'recT_prior2', 'rec2_prior2', 'fum_prior2', 'XPA_prior2', 'XPM_prior2', 'FGA_prior2', 
'FGM_prior2', 'FG50_prior2', 'defSack_prior2', 'defI_prior2', 'defSaf_prior2', 'defFum_prior2', 'defBlk_prior2', 'defT_prior2', 
'defPtsAgainst_prior2', 'defPassYAgainst_prior2', 'defRushYAgainst_prior2', 'defYdsAgainst_prior2', 'defSack_curr_opp', 'defI_curr_opp', 
'defSaf_curr_opp', 'defFum_curr_opp', 'defBlk_curr_opp', 'defT_curr_opp', 'defPtsAgainst_curr_opp', 'defPassYAgainst_curr_opp', 
'defRushYAgainst_curr_opp', 'defYdsAgainst_curr_opp', 'defSack_prior1_opp', 'defI_prior1_opp', 'defSaf_prior1_opp', 'defFum_prior1_opp', 
'defBlk_prior1_opp', 'defT_prior1_opp', 'defPtsAgainst_prior1_opp', 'defPassYAgainst_prior1_opp', 'defRushYAgainst_prior1_opp', 
'defYdsAgainst_prior1_opp', 'pos_PK', 'posRank_PK1', 'posRank_PK2', 'posRank_PK3']]

In [57]:
str(list(X.columns))

"['week', 'age', 'passA_curr', 'passC_curr', 'passY_curr', 'passT_curr', 'passI_curr', 'pass2_curr', 'rushA_curr', 'rushY_curr', 'rushT_curr', 'rush2_curr', 'recC_curr', 'recY_curr', 'recT_curr', 'rec2_curr', 'fum_curr', 'XPA_curr', 'XPM_curr', 'FGA_curr', 'FGM_curr', 'FG50_curr', 'defSack_curr', 'defI_curr', 'defSaf_curr', 'defFum_curr', 'defBlk_curr', 'defT_curr', 'defPtsAgainst_curr', 'defPassYAgainst_curr', 'defRushYAgainst_curr', 'defYdsAgainst_curr', 'gamesPlayed_curr', 'gamesPlayed_prior1', 'passA_prior1', 'passC_prior1', 'passY_prior1', 'passT_prior1', 'passI_prior1', 'pass2_prior1', 'rushA_prior1', 'rushY_prior1', 'rushT_prior1', 'rush2_prior1', 'recC_prior1', 'recY_prior1', 'recT_prior1', 'rec2_prior1', 'fum_prior1', 'XPA_prior1', 'XPM_prior1', 'FGA_prior1', 'FGM_prior1', 'FG50_prior1', 'defSack_prior1', 'defI_prior1', 'defSaf_prior1', 'defFum_prior1', 'defBlk_prior1', 'defT_prior1', 'defPtsAgainst_prior1', 'defPassYAgainst_prior1', 'defRushYAgainst_prior1', 'defYdsAgainst_pr

In [58]:

#load saved model
regressor = load('models/rfmodel_PK1.joblib')

# Run model
y_pred = regressor.predict(X)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = labels
y_pred

# Calculate FANTASY scores
# Define scoring multiplier based on league settings
multiplier = [
    0,0,.04,4,-2,2,.1,.1,6,2,.25,.1,6,2,-2,0,1,0,3,5,1,2,2,2,1.5,6,0,0,0,0,1,1
]
# Define bins for defensive PointsAgainst and YardsAgainst based on MFL scoring categories
binList_defPts = [-5,0,6,13,17,21,27,34,45,59,99]
binList_defYds = [0,274,324,375,425,999]
# Define correlating scores for defensive PointsAgainst and YardsAgainst based on league settings
ptList_defPts = [10,8,7,5,3,2,0,-1,-3,-5]
ptList_defYds = [5,2,0,-2,-5]
# Bin and cut the defensive predictions
y_pred['defPtsBin'] = pd.cut(y_pred['defPtsAgainst'], bins=binList_defPts, include_lowest=True, labels=ptList_defPts)
y_pred['defYdsBin'] = pd.cut(y_pred['defYdsAgainst'], bins=binList_defYds, include_lowest=True, labels=ptList_defYds)
# Merge predictions with header columns so we know the players' position
a_pred = header.merge(y_pred, left_index=True, right_index=True)
# Assign value of zero to all non-defensive players' bins
a_pred.loc[a_pred['pos']!='DF', 'defPtsBin'] = 0
a_pred.loc[a_pred['pos']!='DF', 'defYdsBin'] = 0
# Drop the header columns again
a_pred = a_pred.drop(columns=['id_mfl', 'week','season','team','player','age','sharkRank','adp','pos','KR','PR','RES','posRank','opponent'])
# Create function to apply scoring multiplier
def multer(row):
    return row.multiply(multiplier)
# Apply scoring multiplier to predictions
c = a_pred.apply(multer, axis=1)
c = c.apply(np.sum, axis=1)
c = pd.DataFrame(c, columns=['pred'])

# Merge header columns with predictions
PKdf = header.merge(c, left_index=True, right_index=True)

In [59]:
### DF Predictions
# Read player model and ages
xl2 = player_df.copy()
# Select only one player position
xl2 = xl2.loc[xl2.posRank.isin(['DF1', 'DF2', 'DF3'])]
xl2 = xl2.loc[xl2.pos=='DF']
xl2 = xl2.dropna()
xl2.reset_index(inplace=True, drop=True)

# Select features
X = xl2[features]
header = xl2[[
    'id_mfl',
    'season',
    'week',
    'team',
    'player',
    'age',
    'sharkRank', 
    'adp',
    'KR',
    'PR',
    'RES',
    'pos',
    'posRank',
    'opponent'
]]

# Encode categorical features
X = pd.get_dummies(X, columns = ['pos', 'posRank'])

#load saved model
regressor = load('models/rfmodel_DF1.joblib')

# Run model
y_pred = regressor.predict(X)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = labels
y_pred

# Calculate FANTASY scores
# Define scoring multiplier based on league settings
multiplier = [
    0,0,.04,4,-2,2,.1,.1,6,2,.25,.1,6,2,-2,0,1,0,3,5,1,2,2,2,1.5,6,0,0,0,0,1,1
]
# Define bins for defensive PointsAgainst and YardsAgainst based on MFL scoring categories
binList_defPts = [-5,0,6,13,17,21,27,34,45,59,99]
binList_defYds = [0,274,324,375,425,999]
# Define correlating scores for defensive PointsAgainst and YardsAgainst based on league settings
ptList_defPts = [10,8,7,5,3,2,0,-1,-3,-5]
ptList_defYds = [5,2,0,-2,-5]
# Bin and cut the defensive predictions
y_pred['defPtsBin'] = pd.cut(y_pred['defPtsAgainst'], bins=binList_defPts, include_lowest=True, labels=ptList_defPts)
y_pred['defYdsBin'] = pd.cut(y_pred['defYdsAgainst'], bins=binList_defYds, include_lowest=True, labels=ptList_defYds)
# Merge predictions with header columns so we know the players' position
a_pred = header.merge(y_pred, left_index=True, right_index=True)
# Assign value of zero to all non-defensive players' bins
a_pred.loc[a_pred['pos']!='DF', 'defPtsBin'] = 0
a_pred.loc[a_pred['pos']!='DF', 'defYdsBin'] = 0
# Drop the header columns again
a_pred = a_pred.drop(columns=['id_mfl', 'week','season','team','player','age','sharkRank','adp','pos','KR','PR','RES','posRank','opponent'])
# Create function to apply scoring multiplier
def multer(row):
    return row.multiply(multiplier)
# Apply scoring multiplier to predictions
c = a_pred.apply(multer, axis=1)
c = c.apply(np.sum, axis=1)
c = pd.DataFrame(c, columns=['pred'])

# Merge header columns with predictions
DFdf = header.merge(c, left_index=True, right_index=True)

In [60]:
# Merge all positions' predictions
complete = pd.concat([WRdf, RBdf, QBdf, TEdf, PKdf, DFdf], axis=0)
complete

,id_mfl,season,week,team,player,age,sharkRank,adp,KR,PR,RES,pos,posRank,opponent,pred
0,13164,2022,3.0,LAR,COOPER KUPP,29.0,6.0,5.400000,NO,PR2,NO,WR,WR1,ARI,14.098315
1,13164,2022,4.0,LAR,COOPER KUPP,29.0,6.0,5.400000,NO,PR2,NO,WR,WR1,SFO,13.873552
2,13164,2022,9.0,LAR,COOPER KUPP,29.0,6.0,5.400000,NO,PR2,NO,WR,WR1,TBB,13.306995
3,13164,2022,11.0,LAR,COOPER KUPP,29.0,6.0,5.400000,NO,PR2,NO,WR,WR1,NOS,13.398788
4,13164,2022,12.0,LAR,COOPER KUPP,29.0,6.0,5.400000,NO,PR2,NO,WR,WR1,KCC,13.587991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,0513,2022,10.0,LVR,LAS VEGAS RAIDERS,0.0,983.0,169.960007,NO,NO,NO,DF,DF1,IND,8.423843
540,0513,2022,13.0,LVR,LAS VEGAS RAIDERS,0.0,983.0,169.960007,NO,NO,NO,DF,DF1,LAC,8.567388
541,0513,2022,15.0,LVR,LAS VEGAS RAIDERS,0.0,983.0,169.960007,NO,NO,NO,DF,DF1,NEP,8.440372
542,0513,2022,17.0,LVR,LAS VEGAS RAIDERS,0.0,983.0,169.960007,NO,NO,NO,DF,DF1,SFO,8.479513


In [348]:
# Create summary of annual scores
# analyze weekly df
tPred = complete.groupby('player')['pred'].sum().to_frame()
tPred.reset_index(inplace=True)
info = complete.drop_duplicates(subset=['player', 'pos'], keep='first')
info = info[[
    'id_mfl', 'player', 'age', 'team', 'pos', 'posRank', 'KR', 'PR', 'RES', 'sharkRank', 'adp'
]]

# Merge all predictions  
fullPred = info.merge(tPred, how='left', left_on='player', right_on='player')
fullPred

,id_mfl,player,age,team,pos,posRank,KR,PR,RES,sharkRank,adp,pred
0,13164,COOPER KUPP,29.0,LAR,WR,WR1,NO,PR2,NO,5.0,5.420000,230.955436
1,14836,JUSTIN JEFFERSON,23.0,MIN,WR,WR1,NO,NO,NO,8.0,7.000000,210.078334
2,11675,DAVANTE ADAMS,29.0,LVR,WR,WR1,NO,NO,NO,10.0,12.720000,230.431852
3,14136,DEEBO SAMUEL,26.0,SFO,WR,WR1,NO,NO,NO,22.0,18.799999,237.682198
4,12801,TYREEK HILL,28.0,MIA,WR,WR1,NO,PR1,NO,23.0,22.610001,204.949369
...,...,...,...,...,...,...,...,...,...,...,...,...
800,0515,SEATTLE SEAHAWKS,0.0,SEA,DF,DF1,NO,NO,NO,835.0,3000.000000,138.712584
801,0522,DETROIT LIONS,0.0,DET,DF,DF1,NO,NO,NO,897.0,3000.000000,137.318100
802,0505,NEW YORK JETS,0.0,NYJ,DF,DF1,NO,NO,NO,974.0,3000.000000,138.250927
803,0509,JACKSONVILLE JAGUARS,0.0,JAC,DF,DF1,NO,NO,NO,1118.0,3000.000000,139.717748


In [349]:
# Convert Shark and ADP to point predictions
# Split player df by player pos
qbs = fullPred[fullPred['pos'] == "QB"]
qbs.reset_index(inplace=True, drop=True)
rbs = fullPred[fullPred['pos'] == "RB"]
rbs.reset_index(inplace=True, drop=True)
wrs = fullPred[fullPred['pos'] == "WR"]
wrs.reset_index(inplace=True, drop=True)
tes = fullPred[fullPred['pos'] == "TE"]
tes.reset_index(inplace=True, drop=True)
pks = fullPred[fullPred['pos'] == "PK"]
pks.reset_index(inplace=True, drop=True)
defs = fullPred[fullPred['pos'] == "DF"]
defs.reset_index(inplace=True, drop=True)

In [350]:
# Get standard point projections
point_projections = get_df("point_projections")

# Split point_projection df by player position
qb_proj = point_projections[point_projections['Position'] == "QB"]
qb_proj.reset_index(inplace=True, drop=True)
rb_proj = point_projections[point_projections['Position'] == "RB"]
rb_proj.reset_index(inplace=True, drop=True)
wr_proj = point_projections[point_projections['Position'] == "WR"]
wr_proj.reset_index(inplace=True, drop=True)
te_proj = point_projections[point_projections['Position'] == "TE"]
te_proj.reset_index(inplace=True, drop=True)
pk_proj = point_projections[point_projections['Position'] == "PK"]
pk_proj.reset_index(inplace=True, drop=True)
def_proj = point_projections[point_projections['Position'] == "Def"]
def_proj.reset_index(inplace=True, drop=True)

In [351]:
# Join dfs for current year to point_projection dfs
# Sort current year players by SharkRank
for df in [qbs, rbs, wrs, tes, pks, defs]:
    df.sort_values(by='sharkRank', inplace=True)
    df.reset_index(inplace=True, drop=True)
# Merge dfs
qbs = pd.merge(qbs, qb_proj[['Projection_Relative', 'Projection_Absolute']], how="left", left_index=True, right_index=True)
rbs = pd.merge(rbs, rb_proj[['Projection_Relative', 'Projection_Absolute']], how="left", left_index=True, right_index=True)
wrs = pd.merge(wrs, wr_proj[['Projection_Relative', 'Projection_Absolute']], how="left", left_index=True, right_index=True)
tes = pd.merge(tes, te_proj[['Projection_Relative', 'Projection_Absolute']], how="left", left_index=True, right_index=True)
pks = pd.merge(pks, pk_proj[['Projection_Relative', 'Projection_Absolute']], how="left", left_index=True, right_index=True)
defs = pd.merge(defs, def_proj[['Projection_Relative', 'Projection_Absolute']], how="left", left_index=True, right_index=True)
# Rename columns
for df in [qbs, rbs, wrs, tes, pks, defs]:
    df.rename(columns={'Projection_Relative':'sharkRelative', 'Projection_Absolute':'sharkAbsolute'}, inplace=True)

/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [352]:
# Join dfs for current year to point_projection dfs
# Sort current year players by ADP
for df in [qbs, rbs, wrs, tes, pks, defs]:
    df.sort_values(by='adp', inplace=True)
    df.reset_index(inplace=True, drop=True)
# Merge dfs
qbs = pd.merge(qbs, qb_proj[['Projection_Relative', 'Projection_Absolute']], how="left", left_index=True, right_index=True)
rbs = pd.merge(rbs, rb_proj[['Projection_Relative', 'Projection_Absolute']], how="left", left_index=True, right_index=True)
wrs = pd.merge(wrs, wr_proj[['Projection_Relative', 'Projection_Absolute']], how="left", left_index=True, right_index=True)
tes = pd.merge(tes, te_proj[['Projection_Relative', 'Projection_Absolute']], how="left", left_index=True, right_index=True)
pks = pd.merge(pks, pk_proj[['Projection_Relative', 'Projection_Absolute']], how="left", left_index=True, right_index=True)
defs = pd.merge(defs, def_proj[['Projection_Relative', 'Projection_Absolute']], how="left", left_index=True, right_index=True)
# Rename columns
for df in [qbs, rbs, wrs, tes, pks, defs]:
    df.rename(columns={'Projection_Relative':'adpRelative', 'Projection_Absolute':'adpAbsolute'}, inplace=True)

In [353]:
# Merge all position dfs into one
predictions = pd.concat([qbs, rbs, wrs, tes, pks, defs])
predictions = predictions.sort_values(by=['pred'], ascending=False)
predictions.reset_index(inplace=True, drop=True)
predictions

,id_mfl,player,age,team,pos,posRank,KR,PR,RES,sharkRank,adp,pred,sharkRelative,sharkAbsolute,adpRelative,adpAbsolute
0,5848,TOM BRADY,45.0,TBB,QB,QB1,NO,NO,NO,62.0,61.490002,347.798672,171.646664,329.246664,136.640001,294.240001
1,14779,JUSTIN HERBERT,24.0,LAC,QB,QB1,NO,NO,NO,60.0,30.799999,341.323461,178.046668,335.646667,178.046668,335.646667
2,13116,PATRICK MAHOMES,26.0,KCC,QB,QB1,NO,NO,NO,45.0,27.350000,339.347453,193.409999,351.009999,193.409999,351.009999
3,12620,DAK PRESCOTT,29.0,DAL,QB,QB1,NO,NO,NO,77.0,69.889999,332.541423,136.640001,294.240001,118.080001,275.680001
4,9431,MATTHEW STAFFORD,34.0,LAR,QB,QB1,NO,NO,NO,114.0,83.129997,331.415001,85.600002,243.200002,85.600002,243.200002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800,14555,GUNNER OLSZEWSKI,25.0,PIT,WR,WR3,KR1,PR1,NO,1535.0,3000.000000,23.820542,-99.430000,16.170000,NaN,NaN
801,12394,DEANDRE CARTER,29.0,LAC,WR,WR2,KR1,PR1,NO,1139.0,3000.000000,21.890985,-60.013334,55.586666,-103.250000,12.350000
802,13912,CAMERON BATSON,26.0,ATL,WR,WR2,NO,PR2,NO,1394.0,3000.000000,21.792105,-87.720000,27.880000,NaN,NaN
803,14108,GREG DORTCH,24.0,ARI,WR,WR2,KR2,PR2,NO,3000.0,3000.000000,21.435179,-112.816667,2.783333,-49.436667,66.163333


In [354]:
# Account for punt returners and kick returners
predictions.loc[predictions['KR']=='KR1', 'pred'] = predictions.loc[predictions['KR']=='KR1', 'pred'] + 58.5
predictions.loc[predictions['PR']=='PR1', 'pred'] = predictions.loc[predictions['PR']=='PR1', 'pred'] + 25.5
predictions

,id_mfl,player,age,team,pos,posRank,KR,PR,RES,sharkRank,adp,pred,sharkRelative,sharkAbsolute,adpRelative,adpAbsolute
0,5848,TOM BRADY,45.0,TBB,QB,QB1,NO,NO,NO,62.0,61.490002,347.798672,171.646664,329.246664,136.640001,294.240001
1,14779,JUSTIN HERBERT,24.0,LAC,QB,QB1,NO,NO,NO,60.0,30.799999,341.323461,178.046668,335.646667,178.046668,335.646667
2,13116,PATRICK MAHOMES,26.0,KCC,QB,QB1,NO,NO,NO,45.0,27.350000,339.347453,193.409999,351.009999,193.409999,351.009999
3,12620,DAK PRESCOTT,29.0,DAL,QB,QB1,NO,NO,NO,77.0,69.889999,332.541423,136.640001,294.240001,118.080001,275.680001
4,9431,MATTHEW STAFFORD,34.0,LAR,QB,QB1,NO,NO,NO,114.0,83.129997,331.415001,85.600002,243.200002,85.600002,243.200002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800,14555,GUNNER OLSZEWSKI,25.0,PIT,WR,WR3,KR1,PR1,NO,1535.0,3000.000000,107.820542,-99.430000,16.170000,NaN,NaN
801,12394,DEANDRE CARTER,29.0,LAC,WR,WR2,KR1,PR1,NO,1139.0,3000.000000,105.890985,-60.013334,55.586666,-103.250000,12.350000
802,13912,CAMERON BATSON,26.0,ATL,WR,WR2,NO,PR2,NO,1394.0,3000.000000,21.792105,-87.720000,27.880000,NaN,NaN
803,14108,GREG DORTCH,24.0,ARI,WR,WR2,KR2,PR2,NO,3000.0,3000.000000,21.435179,-112.816667,2.783333,-49.436667,66.163333


In [355]:
# Send predictions to database
# Prepare predictions df
# Build the SQL query that will list columns and datatypes
string1 = [x + " VARCHAR(32)" for x in predictions.columns[:2]] + [
    x + " SMALLINT" for x in predictions.columns[2:3]] + [
    x + " VARCHAR(32)" for x in predictions.columns[3:9]] + [
    x + " FLOAT(8)" for x in predictions.columns[9:]]
string1 = str(string1)
string1 = string1.replace("'", "")
string1 = string1.replace("[", "")
string1 = string1.replace("]", "")
#print(f'CREATE TABLE IF NOT EXISTS predictions({string1})')


# Write the df to the Postgresql database
try:
    # connect to database
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    engine = create_engine(DATABASE_URL)
    cursor = conn.cursor()
    # Create table for schedule
    cursor.execute(f'CREATE TABLE IF NOT EXISTS predictions({string1})')
    conn.commit()
    # Populate table with data
    predictions.to_sql('predictions', engine, if_exists='replace', index = False)
except Exception as error:
    print(error)
finally:
    if conn:
        cursor.close()
        conn.close()